<a href="https://colab.research.google.com/github/vucko77/Bank-Churn-Prediction/blob/main/Vuk_Razgrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread oauth2client sympy


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install --upgrade gspread oauth2client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2


In [ ]:
!pip install gspread oauth2client


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [1]:
#ova treba da go rekonektirash od vreme na vreme
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
json_file_path = '/content/drive/MyDrive/vukrazdrad2-d6c35fdc988b.json'
if os.path.exists(json_file_path):
    print("File found.")
else:
    print("File not found.")


File found.


In [ ]:
json_file_path = '/content/drive/MyDrive/vukrazdrad2-d6c35fdc988b.json'


In [2]:
#2 vtorata Ovaa raboti So Decay factor 10% sekoj den, bez razlika kolku dena ne trenira. Ama dava dupli dates za nekoi igraci.
#Na sekoe prvo startuvanje, koga e prazen destination sheet-ot, manuelno izbrishi gi duplite generirani datumu (kaj nekoi ograci)
#Duplite se generiraat na kraj, posle posledniot vnesen datum (od source file-ot)
#Novi datumi cita- se e ok
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from gspread_dataframe import set_with_dataframe
import time

# Start the timer
start_time = time.time()

# Define the scope
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]

# Provide the path to your service account key file
json_file_path = '/content/drive/MyDrive/vukrazdrad2-d6c35fdc988b.json'
creds = ServiceAccountCredentials.from_json_keyfile_name(json_file_path, scope)
client = gspread.authorize(creds)

# Open the source Google Sheet by URL
source_sheet_url = 'https://docs.google.com/spreadsheets/d/1NgvjR6ToC8TJywy3aYsyQ8g3IoTpXjdu_FJC9CgsJV8/edit?usp=sharing'
source_sh = client.open_by_url(source_sheet_url)
source_worksheet = source_sh.get_worksheet(0)  # Assuming the data is in the first sheet

# Convert the source worksheet to a pandas DataFrame
data = source_worksheet.get_all_records()
df = pd.DataFrame(data)

# Convert 'Start' column to datetime
df['Start'] = pd.to_datetime(df['Start'], format='%d.%m.%Y', errors='coerce')

# Ensure that 'Split' exists and fill with 'no training data' for missing dates
df['Split'] = df['Split'].fillna('no training data')

# Handle duplicates by aggregating (sum or mean)
df = df.groupby(['Player', 'Start', 'Split']).agg({
    'Distance(3+4+5) (m)': 'sum',
    'Total Distance (m)': 'sum',
    'HMLD (m)': 'sum',
    'Sprints count': 'sum',
    'Accelerations': 'sum',
    'Decelerations': 'sum'
}).reset_index()

# Generate a full date range for reindexing
date_range = pd.date_range(start=df['Start'].min(), end=df['Start'].max())

# Reindex each player's data to cover the full date range
df = df.set_index('Start')
df = df.groupby('Player').apply(lambda group: group.reindex(date_range))

# Remove the index levels, if necessary
df.index = df.index.droplevel(0)

# Ensure the 'Player' column is filled correctly
df['Player'] = df['Player'].ffill()

# Reset the index, preserving the 'Start' column and avoiding duplication
df = df.reset_index().rename(columns={'index': 'Start'})

# Fill missing values for specific columns with zeros
df.fillna({'Distance(3+4+5) (m)': 0, 'Total Distance (m)': 0, 'HMLD (m)': 0, 'Sprints count': 0, 'Accelerations': 0, 'Decelerations': 0}, inplace=True)

# Find the first non-zero training day for each player
first_nonzero_dates = df[df[['Distance(3+4+5) (m)', 'Total Distance (m)', 'HMLD (m)', 'Sprints count', 'Accelerations', 'Decelerations']].sum(axis=1) > 0]
first_nonzero_dates = first_nonzero_dates.groupby('Player')['Start'].min()

# Apply decay factor only starting from the first non-zero date for each player
decay_factor = 0.9

def apply_decay_and_ewma(group, decay_factor):
    # Start applying decay factor from the first non-zero date
    first_date = first_nonzero_dates[group['Player'].iloc[0]]
    group = group[group['Start'] >= first_date].copy()

    for col in ['Distance(3+4+5) (m)', 'Total Distance (m)', 'HMLD (m)', 'Sprints count', 'Accelerations', 'Decelerations']:
        for i in range(1, len(group)):
            if pd.isna(group.iloc[i][col]):
                continue
            if group[col].iloc[i] == 0 and group['Player'].iloc[i] == group['Player'].iloc[i-1]:
                group.at[group.index[i], col] = group.at[group.index[i-1], col] * decay_factor

    # Perform vectorized EWMA calculations for each column
    for col in ['Distance(3+4+5) (m)', 'Total Distance (m)', 'HMLD (m)', 'Sprints count', 'Accelerations', 'Decelerations']:
        group[f'EWMA3_{col}'] = group[col].ewm(span=3, adjust=False).mean()
        group[f'EWMA7_{col}'] = group[col].ewm(span=7, adjust=False).mean()
        group[f'EWMA21_{col}'] = group[col].ewm(span=21, adjust=False).mean()
        group[f'EWMA28_{col}'] = group[col].ewm(span=28, adjust=False).mean()

    return group

print(" -1- ")
# Apply the decay and EWMA function group-wise
df = df.groupby('Player').apply(lambda group: apply_decay_and_ewma(group, decay_factor))
print(" -2- ")

# Open the destination Google Sheet by URL
destination_sheet_url = 'https://docs.google.com/spreadsheets/d/1iMfMTTO5WGm7tLtDp_-_eEpBgKc7N-2oeELBMk3cbOs/edit?usp=sharing'
destination_sh = client.open_by_url(destination_sheet_url)
destination_worksheet = destination_sh.get_worksheet(0)  # Assuming the data should be written to the first sheet

# Load existing data from the destination sheet
existing_data = destination_worksheet.get_all_records()
print(" -4- ")
existing_df = pd.DataFrame(existing_data)
print(" -5 ")
# Convert 'Start' in the destination to datetime if not empty AND if 'Start' column exists
if not existing_df.empty and 'Start' in existing_df.columns:
    existing_df['Start'] = pd.to_datetime(existing_df['Start'], errors='coerce')

# If the destination sheet is not empty, filter out already processed rows
if not existing_df.empty and 'Start' in df.columns and 'Start' in existing_df.columns:
    max_existing_date = existing_df['Start'].max()
    new_data = df[df['Start'] > max_existing_date]
else:
    new_data = df.copy()

# Reorder columns for exporting to match the destination sheet
columns_to_export = [
    'Player', 'Split', 'Start', 'Distance(3+4+5) (m)', 'Total Distance (m)',
    'HMLD (m)', 'Sprints count', 'Accelerations', 'Decelerations',
    'EWMA3_Distance(3+4+5) (m)', 'EWMA7_Distance(3+4+5) (m)',
    'EWMA21_Distance(3+4+5) (m)', 'EWMA28_Distance(3+4+5) (m)',
    'EWMA3_Total Distance (m)', 'EWMA7_Total Distance (m)',
    'EWMA21_Total Distance (m)', 'EWMA28_Total Distance (m)',
    'EWMA3_HMLD (m)', 'EWMA7_HMLD (m)', 'EWMA21_HMLD (m)',
    'EWMA28_HMLD (m)', 'EWMA3_Sprints count', 'EWMA7_Sprints count',
    'EWMA21_Sprints count', 'EWMA28_Sprints count',
    'EWMA3_Accelerations', 'EWMA7_Accelerations',
    'EWMA21_Accelerations', 'EWMA28_Accelerations',
    'EWMA3_Decelerations', 'EWMA7_Decelerations',
    'EWMA21_Decelerations', 'EWMA28_Decelerations'
]

# Ensure 'Split' and 'Start' columns exist before exporting
if 'Split' not in new_data.columns:
    new_data['Split'] = 'no training data'
if 'Start' not in new_data.columns:
    new_data['Start'] = pd.NaT

new_data = new_data[columns_to_export]

if not new_data.empty:
    print(f"New data found. Processing {len(new_data)} new rows.")

    # Add headers only if the destination sheet is empty
    start_row = len(existing_df) + 2 if not existing_df.empty else 1

    set_with_dataframe(destination_worksheet, new_data, row=start_row, include_index=False, include_column_header=existing_df.empty)

    print("New data has been successfully processed and appended to the destination sheet.")
else:
    print("No new data to process.")

# Stop the timer and print the execution time
end_time = time.time()
execution_time = end_time - start_time
print(f"Script executed in {execution_time:.2f} seconds.")


 -1- 
 -2- 
 -4- 
 -5 
New data found. Processing 24 new rows.
New data has been successfully processed and appended to the destination sheet.
Script executed in 5.71 seconds.


In [3]:
#1 prvata EWMA
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
from gspread_dataframe import set_with_dataframe
import time

# Start the timer
start_time = time.time()

# Define the scope
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]

# Provide the path to your service account key file
json_file_path = '/content/drive/MyDrive/vukrazdrad2-d6c35fdc988b.json'
creds = ServiceAccountCredentials.from_json_keyfile_name(json_file_path, scope)
client = gspread.authorize(creds)

# Open the source Google Sheet by URL
source_sheet_url = 'https://docs.google.com/spreadsheets/d/1NgvjR6ToC8TJywy3aYsyQ8g3IoTpXjdu_FJC9CgsJV8/edit?usp=sharing'
source_sh = client.open_by_url(source_sheet_url)
source_worksheet = source_sh.get_worksheet(0)  # Assuming the data is in the first sheet

# Convert the source worksheet to a pandas DataFrame
data = source_worksheet.get_all_records()
df = pd.DataFrame(data)

# Convert 'Start' column to datetime
df['Start'] = pd.to_datetime(df['Start'], errors='coerce')

# EWMA Constants
const3 = 0.5
const7 = 0.25
const28 = 0.07
const21 = 0.09

# Updated Columns to calculate EWMAs for
columns_to_calculate = ['Distance(3+4+5) (m)', 'Total Distance (m)', 'HMLD (m)', 'Sprints count', 'Accelerations', 'Decelerations']

# Sorting data by Player and Start date
df.sort_values(by=['Player', 'Start'], ascending=[True, True], inplace=True)

# Perform vectorized EWMA calculations for each column
for col in columns_to_calculate:
    df[f'EWMA3_{col}'] = df.groupby('Player')[col].transform(lambda x: x.ewm(alpha=const3).mean())
    df[f'EWMA7_{col}'] = df.groupby('Player')[col].transform(lambda x: x.ewm(alpha=const7).mean())
    df[f'EWMA21_{col}'] = df.groupby('Player')[col].transform(lambda x: x.ewm(alpha=const21).mean())
    df[f'EWMA28_{col}'] = df.groupby('Player')[col].transform(lambda x: x.ewm(alpha=const28).mean())

# Open the destination Google Sheet by URL
destination_sheet_url = 'https://docs.google.com/spreadsheets/d/19mZX-CpfxoE00-gNmcK2FQArF8Lwq-fahWal6vi82WI/edit?usp=sharing'
destination_sh = client.open_by_url(destination_sheet_url)
destination_worksheet = destination_sh.get_worksheet(0)  # Assuming the data should be written to the first sheet

# Load existing data from the destination sheet
existing_data = destination_worksheet.get_all_records()
existing_df = pd.DataFrame(existing_data)

# If the destination sheet is not empty, filter out already processed rows
if not existing_df.empty:
    existing_df['Start'] = pd.to_datetime(existing_df['Start'], errors='coerce')
    max_existing_date = existing_df['Start'].max()
    new_data = df[df['Start'] > max_existing_date]
else:
    new_data = df.copy()

if not new_data.empty:
    print(f"New data found. Processing {len(new_data)} new rows.")

    # Check if the destination is empty and add headers if it is
    start_row = len(existing_df) + 2 if not existing_df.empty else 1  # Start row for new data (considering 1-indexed and header row)

    set_with_dataframe(destination_worksheet, new_data, row=start_row, include_index=False, include_column_header=existing_df.empty)

    print("New data has been successfully processed and appended to the destination sheet.")
else:
    print("No new data to process.")

# Stop the timer and print the execution time
end_time = time.time()
execution_time = end_time - start_time
print(f"Script executed in {execution_time:.2f} seconds.")


<ipython-input-3-ac40e37246ce>:33: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Start'] = pd.to_datetime(df['Start'], errors='coerce')


New data found. Processing 20 new rows.
New data has been successfully processed and appended to the destination sheet.
Script executed in 2.33 seconds.
